This is a brief introduction to Pandas. It is definitely not the first. Highly recommended resources for further reading include:   
- **Tutorials from Stern's "Dealing with Data" course**:     https://github.com/ipeirotis/dealing_with_data 
- ** Python for Data Analysis**, by Wes McKinney (the creator of Pandas):    
http://shop.oreilly.com/product/0636920023784.do
- ** StackOverflow **:     
http://stackoverflow.com/questions/tagged/pandas
- ** The Pandas documentation **:     
http://pandas.pydata.org/pandas-docs/stable/
- ** Chris Albon's blog**, which has consistently useful code snippets:
http://chrisalbon.com/#Python

# 0. Setup
First things first: let's load our libraries.

In [ ]:
import pandas as pd               # For working with dataframes
import numpy as np                # For numeric and scientific computations
import matplotlib.pyplot as plt   # For graphing
import urllib                     # For downloading content from a URL
import zipfile                    # For unzipping files

%matplotlib inline

Did you install pandas?

In [ ]:
# We can access the command line with an exclamation point, like this:
!pip install pandas

Now, download the file from the web

In [ ]:
source = "http://nhts.ornl.gov/2009/download/TripChainingCSV.zip"
urllib.urlretrieve(source, "chntrp09.zip")

And, unzip it

In [ ]:
zip_ref = zipfile.ZipFile("chntrp09.zip", 'r')
zip_ref.extractall("chntrp09")
zip_ref.close()

OK, now, we can get started.

# 1. Importing CSV data

We read in a csv file like this:

In [ ]:
trip_data = pd.read_csv("chntrp09.csv")

What happened?

In [ ]:
trip_data.head()

## a. Limit the number of rows read

In [ ]:
trip_data = pd.read_csv("chntrp09.csv", nrows = 1)
trip_data.head()

## b. Choose which columns to import

In [ ]:
trip_data = pd.read_csv("chntrp09.csv", usecols=['HOUSEID', 'PERSONID'] )
trip_data.head()

## c. Import piecewise
This is often useful if you have a plan for collapsing data in mind.   
For example, you might be counting by groups, so you can count by group within each chunk, and then sum over all chunks.

In [ ]:
counter = 0

for trip_data_chunk in pd.read_csv("chntrp09.csv", 
                                   usecols=['HOUSEID', 'PERSONID'],
                                  chunksize=300000):
    
    print "Reading chunk", counter
    counter +=1

In [ ]:
### How would you read the first 2,000 rows of the 'TOUR' and 'TOUR_SEG' columns?

<font color = "white"> trip_data = pd.read_csv("chntrp09.csv", nrows = 2000, usecols=['TOUR', 'TOUR_SEG'])

# 2. Inspecting the data

In [ ]:
trip_data = pd.read_csv("chntrp09.csv")

## a. How many rows and columns?

In [ ]:
trip_data.shape

## b. What are the columns?

In [ ]:
trip_data.columns

## c. What are the data types of the columns?

In [ ]:
trip_data.dtypes

## d.  How many entries in the columns?

In [ ]:
trip_data.count()

## e. A brief summary of the columns?

In [ ]:
trip_data.describe()

# 3. Browsing and selecting data

In [ ]:
trip_data.head()

## a. Select columns
### Single

In [ ]:
trip_data.HOUSEID.head()

In [ ]:
trip_data['HOUSEID'].head()

### Multiple

In [ ]:
trip_data[['HOUSEID','TOUR']].head()

## c. Select rows
### Single

In [ ]:
trip_data.loc[0]

### Mutiple

In [ ]:
trip_data.loc[[0,6]]

In [ ]:
trip_data.loc[0:6]

## e. Select cells
### Single

In [ ]:
trip_data.loc[3, "TDTRPNUM"]

### Multiple

In [ ]:
trip_data.loc[3:7, "TDTRPNUM":'TOUR']

## f. Select subset

In [ ]:
trip_data[trip_data.TOUR==2].head()

In [ ]:
trip_data[(trip_data.TOUR==2) & (trip_data.STOPS==0)].head()

In [ ]:
trip_data.loc[trip_data.HOUSEID == 20000231, 'WTTRDFIN']

In [ ]:
### How would you select the first 20 rows of 'WTTRDFIN' and 'PERSONID'?

<font color="white"> trip_data[:20, ['WTTRDFIN', 'PERSONID']] 

# 4. Cleaning the data

## a. Edit column names
### Systematically

In [ ]:
trip_data.columns = [c.lower() for c in trip_data.columns]
trip_data.head()

### Manually

In [ ]:
trip_data.rename(columns={'houseid': 'house_id',
                         'personid': 'person_id'}, inplace = True)
trip_data.head()

## b. Change column values

In [ ]:
trip_data.trpcnt.describe()

In [ ]:
trip_data.loc[trip_data.trpcnt==21, 'trpcnt'] = 999

In [ ]:
trip_data.trpcnt.describe()

## c. Generate new columns

In [ ]:
trip_data['trpcnt_recoded'] = np.where(trip_data.trpcnt>10, "A lot", "A little")

In [ ]:
pd.crosstab(trip_data['trpcnt'], trip_data['trpcnt_recoded'])

In [ ]:
trip_data['stops100'] = trip_data.stops * 100
trip_data['stops3'] = trip_data.stops ** 2


trip_data[['stops', 'stops100', 'stops3']].head(20)

## d. Insert missing values

In [ ]:
trip_data.loc[trip_data.stops==0,'stops'] = np.nan

In [ ]:
trip_data.stops.head()

### !!!! Caution: Missing values get weird
#### Nulls are not equal

In [ ]:
trip_data.loc[0].stops

In [ ]:
trip_data.loc[1].stops

In [ ]:
trip_data.loc[0].stops == trip_data.loc[1].stops

In [ ]:
pd.isnull(trip_data.loc[1].stops)

#### For this reason, we need special option to select them

In [ ]:
trip_data[trip_data.stops==np.nan]

In [ ]:
trip_data[trip_data.stops.isnull()].head()

#### They don't appear to be ordered

In [ ]:
trip_data.loc[0].stops>0

In [ ]:
trip_data.loc[0].stops<0

In [ ]:
trip_data.loc[0].stops==0

# 5. Reshaping the data

## a. Calculate group characteristics

In [ ]:
trip_data.groupby(['house_id', 'person_id']).mean().head()

## b. Pivot

In [ ]:
trip_data.groupby(['house_id', 'person_id']).mean()[['tdtrpnum', 'trpcnt']].head()

In [ ]:
trip_data.groupby(['house_id', 'person_id']).mean()[['tdtrpnum', 'trpcnt']].unstack().head()

## c. Join

In [ ]:
trip_data[['house_id','person_id', 'tdtrpnum']].merge(trip_data[['house_id','person_id', 'trpcnt',  'tdtrpnum']],
                                                    on=['house_id','person_id'], suffixes=["_l", "_r"]).head()

# 6. Inspecting the data 

In [ ]:
trip_data.head()

## a. Get unique values

In [ ]:
trip_data.tour.unique()

## b. Count values

In [ ]:
trip_data.stops.value_counts()

In [ ]:
### How would you count unique values of stops if trpcnt_recoded is "A lot"?

<font color="white"> trip_data.loc[trip_data.trpcnt_recoded=="A lot", "stops"].value_counts()

## b. Plot
### Histogram

In [ ]:
trip_data.stops.hist(range=[0,10], bins=10)

In [ ]:
trip_data.sample(5000).plot(kind='scatter', x='tdtrpnum', y='tour', alpha=0.1)

In [ ]:
trip_data.sample(5000).plot(kind='scatter', x='house_id', y='wttrdfin', alpha=0.1, ylim=0)

In [ ]:
trip_data['little'] = np.where(trip_data.trpcnt_recoded=="A little", 1, 0)
trip_data['lot'] = np.where(trip_data.trpcnt_recoded=="A lot", 1, 0)

In [ ]:
trip_data.groupby('tourtype').count()['house_id'].plot(kind='bar', width=1, alpha=0.5, edgecolor='white') #[['little', 'lot']].plot(kind='bar')

In [ ]:
trip_data.groupby('tour').mean()['stops'].plot(kind='line', legend='stops')
trip_data.groupby('tour').mean()['trpcnt'].plot(kind='bar', legend ='trpcnt', alpha=0.2, edgecolor='white', width=1)

In [ ]:
trip_data.groupby('stops').mean()['stops100'].plot(kind='line')
trip_data.groupby('stops').mean()['stops3'].plot(kind='line')